In [2]:
import cv2
import pandas as pd
import numpy as np

In [3]:
# Read the CSV file
df = pd.read_csv('../Data/2025-Coral/FullData.csv')
df

,Image,Center_X,Center_Y,Width,Height,Angle,x_position,y_position,angle
0,0,14,103,36,28,90,-1.699896,4.546147,0.0
1,1,12,103,37,25,90,-1.699896,4.546147,5.0
2,2,11,104,38,22,90,-1.699896,4.546147,10.0
3,3,9,104,37,18,90,-1.699896,4.546147,15.0
4,4,9,104,37,18,90,-1.699896,4.546147,20.0
...,...,...,...,...,...,...,...,...,...
198931,198931,1266,41,29,25,90,4.250104,3.896147,155.0
198932,198932,1265,41,29,28,90,4.250104,3.896147,160.0
198933,198933,1265,40,32,26,8,4.250104,3.896147,165.0
198934,198934,1264,40,33,26,8,4.250104,3.896147,170.0


When a new image is captured we need to run the same algorithm we ran to create the data

In [10]:
# Detect game pice by color
# And create a rect around the detection
def find_oriented_bounding_rect(image, lower_color, upper_color):
    """    Detects the largest contour of a specified color in the image and returns its bounding rectangle.
    Args:
        image (numpy.ndarray): The input image in which to find the contour.
        lower_color (tuple): The lower bound of the color in HSV format.
        upper_color (tuple): The upper bound of the color in HSV format.
    Returns:
        tuple: A tuple containing the coordinates (x, y) and dimensions (width, height) of the bounding rectangle.
    """
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Combine masks to get only red pixels
    red_mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Threshold the mask to create a binary image
    _, thresh = cv2.threshold(red_mask, 127, 255, cv2.THRESH_BINARY)

    # Define the kernel for the morphological operation
    # ChatGPT gave me this part of the code, I have no idea how it works
    kernel = np.ones((5, 5), np.uint8)

    # Perform morphological opening to remove noise
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Perform morphological closing to close small holes
    # This really helps to cleanup the mask
    processed_img = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(processed_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for cont in contours:
        oriented_rect = cv2.minAreaRect(cont)
        box = cv2.boxPoints(oriented_rect)
        box = np.int0(box)
        cv2.drawContours(image, [box], 0, (0, 255, 255), 2)

    contour = max(contours, key=cv2.contourArea)
    
    # Get oriented bounding rectangle
    oriented_rect = cv2.minAreaRect(cont)
    box = cv2.boxPoints(oriented_rect)
    box = np.intp(box)

    (center_x, center_y), (width, height), angle = np.intp(oriented_rect)

    cv2.drawContours(image, contour, -1, (0, 255, 0), 3)
    cv2.drawContours(image, [box], 0, (0, 255, 255), 2)
    cv2.imshow("Frame", image)
    cv2.imshow("Mask", processed_img)
    cv2.waitKey(0)

    return (center_x, center_y, width, height, angle)

Find similar data points inside our data base to find the real world position

In [6]:
def find_matching_rows(df, target, start_tol=2, max_tol=15, step=2):
    """
    Find rows in the dataframe that match the target values within a dynamically
    increasing tolerance. If no rows are found for a very small tolerance, the
    tolerance will be increased until at least one row is found or the maximum
    tolerance is reached.

    Parameters:
        df (pd.DataFrame): DataFrame with the columns.
        target (dict): Target values for each column.
        start_tol (int): Starting tolerance measured in pixels.
        max_tol (int): Maximum allowed tolerance measured in pixels.
        step (int): Increment to increase tolerance on each iteration measured in pixels.

    Returns:
        filtered_df (pd.DataFrame): DataFrame with matching rows.
        used_tol (int): Tolerance at which the matching rows were found.
    """
    
    tolerance = start_tol
    while tolerance <= max_tol:
        # Create mask using np.isclose for each column
        mask = (
            np.isclose(df['Center_X'], target['Center_X'], atol=tolerance) &
            np.isclose(df['Center_Y'], target['Center_Y'], atol=tolerance) &
            np.isclose(df['Width'], target['Width'], atol=tolerance) &
            np.isclose(df['Height'], target['Height'], atol=tolerance) &
            np.isclose(df['Angle'], target['Angle'], atol=tolerance)
        )
        filtered_df = df[mask]
        
        # Check if any row is found
        if not filtered_df.empty:
            print(f"Found rows with tolerance: {tolerance}")
            return filtered_df, tolerance
        
        # Increase the tolerance and try again
        tolerance += step

    # No rows found within maximum tolerance
    print("No rows found within the max tolerance.")
    return df.iloc[[]], tolerance  # Return an empty DataFrame

Main function running on the Raspberry-PI

In [ ]:
targetRes = (1280, 720)  # Target resolution for the image

# Capture the image using the connected camera (For testing purposes load an image from disk)
image = cv2.imread("will/be/added/later.jpg")

# Resize the image to the target size
image = cv2.resize(image, targetRes)

# The low bounds, [h, s, v]
low_bounds = np.array([9, 35, 0])
# The top bounds, [h, s, v]
top_bounds = np.array([31, 255, 255])

# Detect the closest game piece in the image, using ml model or a simple color filter
oriented_rect = find_oriented_bounding_rect(image, low_bounds, top_bounds)
cv2.destroyAllWindows()
c_x, c_y, w, h, a = oriented_rect

target = {
    'Center_X': c_x,
    'Center_Y': c_y,
    'Width': w,
    'Height': h,
    'Angle': a
}

# Assuming df is the pandas DataFrame
result_df, used_tolerance = find_matching_rows(df, target, start_tol=25, max_tol=40, step=3)
result_df

In [ ]:
coral_position = [result_df['x_position'].mean(), result_df['y_position'].mean()]
print(coral_position)
print(f"Coral angle: {result_df['Angle'].mean()} degrees")